In [1]:
import numpy as np
import tensorflow as tf
import os
import utils.mem_utils as mem_utils
import utils.get_data as get_data

In [5]:
def sigmoid(a_in, beta=1):
  """Hyperbolic tangent non-linearity"""
  a_out = tf.subtract(tf.multiply(2.0, tf.divide(1.0,
    tf.add(1.0, tf.exp(tf.multiply(-beta, a_in))))), 1.0)
  return a_out

In [6]:
def memristorize(u_in, memrister_std_eps, memristor_type=None):
  if memristor_type is None:
    return u_in
  elif memristor_type == "gauss":
    get_channel_data = get_data.get_gauss_data
  elif memristor_type == "rram":
    get_channel_data = get_data.get_rram_data
  elif memristor_type == "pcm":
    get_channel_data = get_data.get_pcm_data
  else:
    assert False, ("memristor_type must be None, 'rram', 'gauss', or 'pcm'")
  memristor_data_loc = os.path.expanduser("~")+"/CAE_Project/CAEs/data/Partial_Reset_PCM.pkl", 
  n_mem = 448
  mem_v_min = -1.0
  mem_v_max = 1.0
  u_in_shape = tf.shape(u_in)
  (vs_data, mus_data, sigs_data,
    orig_VMIN, orig_VMAX, orig_RMIN,
    orig_RMAX) = get_channel_data(memristor_data_loc, n_mem, num_ext=5, norm_min=mem_v_min, norm_max=mem_v_max)
  v_clip = tf.clip_by_value(u_in, clip_value_min=mem_v_min, clip_value_max=mem_v_max)
  r = mem_utils.memristor_output(v_clip, memristor_std_eps, vs_data, mus_data, sigs_data,
    interp_width=np.array(vs_data[1, 0] - vs_data[0, 0]).astype('float32'))
  u_out = tf.reshape(r, shape=u_in_shape, name="mem_r")
  return u_out

In [9]:
n_mem = 448
memristor_std_eps = np.random.standard_normal((1, n_mem)).astype(np.float32)
input_voltages = np.load("/home/dpaiton/image_rram_encoding.npz")["data"]
  
x = tf.placeholder(tf.float32, shape=[n_mem,], name="encoded_img")
x_sig = sigmoid(x, beta=1.0)

u_out = memristorize(x_sig, memristor_std_eps, memristor_type="rram")
u_out_nosig = memristorize(x, memristor_std_eps, memristor_type="rram")

with tf.Session() as sess:
  u_eval, u_nosig_eval, input_sig = sess.run([u_out, u_out_nosig, x_sig], feed_dict={x:input_voltages})

save_path = "/home/dpaiton/"
np.savez(save_path+"u_sig.npz", data=u_eval)
np.savez(save_path+"u_nosig.npz", data=u_nosig_eval)

TypeError: invalid file: ('/home/dpaiton/CAE_Project/CAEs/data/Partial_Reset_PCM.pkl',)

In [2]:
import pandas as pd
import os
memristor_data_loc = os.path.expanduser("~")+"/CAE_Project/CAEs/data/Partial_Reset_PCM.pkl", 
Vs,_ = pd.read_pickle(memristor_data_loc)

TypeError: invalid file: ('/home/dpaiton/CAE_Project/CAEs/data/Partial_Reset_PCM.pkl',)